In [ ]:
import tensorflow
from tensorflow import keras
from keras import layers, models, regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

In [ ]:
# giving directory path to data set
dataset_path='/content/drive/MyDrive/train'

In [ ]:
image_size=(65,65)
# data_generator = ImageDataGenerator(rescale=1./255)
data_generator=ImageDataGenerator(rescale=1./255,rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.2)
train_dataset=data_generator.flow_from_directory(dataset_path,target_size=image_size,batch_size=32,class_mode='binary',shuffle=True,subset="training")
validation_dataset=data_generator.flow_from_directory(dataset_path,target_size=image_size,batch_size=32,class_mode='binary',shuffle=True,subset="validation")

In [ ]:
train_dataset.class_indices

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',padding="same"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu',padding="same"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu',padding="same"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2,activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
earlystopping=callbacks.EarlyStopping(monitor="accuracy",mode="max",patience=3,restore_best_weights=True)

In [ ]:
model.fit(train_dataset,epochs=15,batch_size=32,validation_data=validation_dataset,callbacks=[earlystopping])